In [3]:
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient
import os
from tika import parser
import pdfplumber
import io

storage_account_name = 'chatgptv2stn'
container_name = 'chatgpt-ctn'
resource_group_name = 'chatgptGp'
    


In [4]:
# Extract text from a PDF file
def tika_parser(blob_data):
    try:
        with io.BytesIO(blob_data) as pdf_file:
            # Try to extract text using Tika parser
            try:
                parsed_pdf = parser.from_buffer(pdf_file)
                text = parsed_pdf['content']
                print(f"Successfully extracted {len(text)} characters from PDF using Tika")
                return text
            except:
                pass

            # If Tika fails, try to extract text using pdfplumber
            try:
                with pdfplumber.load(pdf_file) as pdf:
                    pages = pdf.pages
                    text = "\n".join([page.extract_text() for page in pages])
                    print(f"Successfully extracted {len(text)} characters from PDF using pdfplumber")
                    return text
            except:
                
                pass

            # If both Tika and pdfplumber fail, return None
            print("Failed to extract text from PDF using Tika or pdfplumber")
            return None
    except:
        print("Error reading PDF file from memory")
        return None


In [35]:
[item.name for item in ContainerClient.from_connection_string(conn_str=storage_connection_string, container_name=container_name).list_blobs()][0]

"/src/AcademicJournal/anthropology/Annaud/De l'intestin aux testicules Substances, humeurs et alliance tikar - Cameroun central.pdf"

In [5]:

storage_connection_string = os.popen(f"az storage account show-connection-string -g {resource_group_name} -n {storage_account_name} --query connectionString").read().strip()


In [9]:
def extract_text_from_container(resource_group_name, storage_account_name, container_name, list_of_pdf_uploaded =[]):
    storage_connection_string = os.popen(f"az storage account show-connection-string -g {resource_group_name} -n {storage_account_name} --query connectionString").read().strip()
    blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    return [(item.name,tika_parser(BlobServiceClient.from_connection_string(storage_connection_string).get_blob_client(container=container_name,blob= item.name).download_blob().content_as_bytes()))\
                for item in \
                ContainerClient.from_connection_string(conn_str=storage_connection_string, container_name=container_name).list_blobs()  \
                if item.name not in list_of_pdf_uploaded]
                

In [10]:
a = extract_text_from_container(resource_group_name, storage_account_name, container_name, list_of_pdf_uploaded =[])

Successfully extracted 50645 characters from PDF using Tika
Successfully extracted 53836 characters from PDF using Tika
Successfully extracted 43515 characters from PDF using Tika
Successfully extracted 40730 characters from PDF using Tika
Successfully extracted 75278 characters from PDF using Tika
Successfully extracted 61900 characters from PDF using Tika
Successfully extracted 32119 characters from PDF using Tika
Successfully extracted 71598 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 41671 characters from PDF using Tika
Successfully extracted 25369 characters from PDF using Tika
Successfully extracted 76719 characters from PDF using Tika
Successfully extracted 127 characters from PDF using Tika
Successfully extracted 47623 characters from PDF using Tika
Successfully extracted 38913 characters from PDF using Tika
Successfully extracted 39577 characters from PDF using Tika
Successfully extracted 42013 characters from 

In [11]:
a

[("/src/AcademicJournal/anthropology/Annaud/De l'intestin aux testicules Substances, humeurs et alliance tikar - Cameroun central.pdf",
  '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDe l\'intestin aux testicules: Substances, humeurs et alliance tikar (Cameroun central)\n\n\nEHESS\n\nDe l\'intestin aux testicules: Substances, humeurs et alliance tikar (Cameroun central)\nAuthor(s): Mathilde Annaud\nSource: L\'Homme, No. 154/155, Question de parenté (Apr. - Sep., 2000), pp. 357-372\nPublished by: EHESS\nStable URL: http://www.jstor.org/stable/40661781 .\n\nAccessed: 10/06/2014 03:23\n\nYour use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at .\nhttp://www.jstor.org/page/info/about/policies/terms.jsp\n\n .\nJSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of\ncontent in a trusted digital archive. We use information technology and tools to increase

In [7]:

[(item.name,tika_parser(BlobServiceClient.from_connection_string(storage_connection_string).get_blob_client(container=container_name,blob= item.name).download_blob().content_as_bytes()))\
  for item in \
 ContainerClient.from_connection_string(conn_str=storage_connection_string, container_name=container_name).list_blobs() 
]

Successfully extracted 50645 characters from PDF using Tika
Successfully extracted 53836 characters from PDF using Tika
Successfully extracted 43515 characters from PDF using Tika
Successfully extracted 40730 characters from PDF using Tika
Successfully extracted 75278 characters from PDF using Tika
Successfully extracted 61900 characters from PDF using Tika
Successfully extracted 32119 characters from PDF using Tika
Successfully extracted 71598 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 41671 characters from PDF using Tika
Successfully extracted 25369 characters from PDF using Tika
Successfully extracted 76719 characters from PDF using Tika
Successfully extracted 127 characters from PDF using Tika
Successfully extracted 47623 characters from PDF using Tika
Successfully extracted 38913 characters from PDF using Tika
Successfully extracted 39577 characters from PDF using Tika
Successfully extracted 42013 characters from 

[("/src/AcademicJournal/anthropology/Annaud/De l'intestin aux testicules Substances, humeurs et alliance tikar - Cameroun central.pdf",
  '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDe l\'intestin aux testicules: Substances, humeurs et alliance tikar (Cameroun central)\n\n\nEHESS\n\nDe l\'intestin aux testicules: Substances, humeurs et alliance tikar (Cameroun central)\nAuthor(s): Mathilde Annaud\nSource: L\'Homme, No. 154/155, Question de parenté (Apr. - Sep., 2000), pp. 357-372\nPublished by: EHESS\nStable URL: http://www.jstor.org/stable/40661781 .\n\nAccessed: 10/06/2014 03:23\n\nYour use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at .\nhttp://www.jstor.org/page/info/about/policies/terms.jsp\n\n .\nJSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of\ncontent in a trusted digital archive. We use information technology and tools to increase

In [18]:
[item.name for item in blob_list]

[]